# How fast are list comprehensions?

List comprehensions give python developers a concise way to create lists from other lists or iterables. They are easy to read, execute quickly, and feel right at home in python. But this isn't the only way to create a new list from a sequence. Like many languages, python supports lambda functions, or anonymous functions. There are ways to use familiar functional programming calls, such as `map`, in combination with lambdas to generate new lists. Let's look at what they both are and then see if either have a performance advantage.

## List comprehensions

List comprehensions are easily recognized as an expression inside of list brackets. Below is a simple example of a list comprehension to create a list of cubes.

In [93]:
[x**3 for x in range(0,5)]

[0, 1, 8, 27, 64]

## Functional approach

We can create the same list using functional concepts. Note, while I'm calling these functional, there is no real guard stopping you from introducing side effects.

In [84]:
list(map(lambda x: x**3, range(0,5)))

[0, 1, 8, 27, 64]

## What happend to for loops?

Of course this can all be done in for loops still, as seen here:

In [85]:
cubes = []
for x in range(0,5):
    cubes.append(x**3)
cubes

[0, 1, 8, 27, 64]

## Is one faster than the other?

We can put all three of those into a timing function to see which is quicker, and we will do that, but there is an interesting issue that we should see will happen already. What's that `list()` call we have in the functional approach? If we don't already have a list, what does `map()` actually return? Let's first just check the performance of these without any changes then we can answer those questions.

In [109]:
thousand = list(range(0, 1000))

def comprehension():
    [x**3 for x in thousand]

def map_lambda():
    list(map(lambda x: x**3, thousand))

def for_loop():
    cubes = []
    for x in thousand:
        cubes.append(x**3)

%timeit comprehension()
%timeit map_lambda()
%timeit for_loop()

242 µs ± 1.98 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
270 µs ± 3.15 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
273 µs ± 2.92 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In this experiment, it looks like list comprehennsions are faster than your other options. Just before, we asked about the `list()` call in our functional approach. `Map()` returns an iterator, not actually a list. An iterator is an object that has a `next` method, which returns the next item in the sequence. You can iterate through an iterator! What if we remove the `list()` call and time it now:

In [110]:
thousand = list(range(0, 1000))

def comprehension():
    [x**3 for x in thousand]

def map_lambda():
    map(lambda x: x**3, thousand)

def for_loop():
    cubes = []
    for x in thousand:
        cubes.append(x**3)

%timeit comprehension()
%timeit map_lambda()
%timeit for_loop()

253 µs ± 3.58 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
239 ns ± 6.71 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
291 µs ± 7.91 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Now `map_lambda` ran exponentially faster than any of the other options. But know that we're cheating a bit, we have an iterator, not a list. But if we are willing to cut corners like this, doesn't this feel a bit like a generator?

## Generators

Generators are functions that behave like iterators, and we even have a notation called generator expressions that looks a lot like list comprehesions.

In [111]:
(x**3 for x in range(0,5))

<generator object <genexpr> at 0x10eeef5e8>

Just switching from brackets to parentheses and now we have a generator. This returns an itterator just as our functional approach did, but this seems like a much easier synntax. We returned a `generator object`, not a list. This is an iterator, and what if we wanted a list from this iterator?

In [112]:
list((x**3 for x in range(0,5)))

[0, 1, 8, 27, 64]

We brought back our `list()` call, and made a list out of a generator. You can even think of list comprehensions this way, a passing the output of a generator into `list()`.

Let's go back to our speed test but include generators this time.

In [115]:
thousand = list(range(0, 1000))

def comprehension():
    [x**3 for x in thousand]

def map_lambda():
    list(map(lambda x: x**3, thousand))

def generator():
    list((x**3 for x in thousand))

def for_loop():
    cubes = []
    for x in thousand:
        cubes.append(x**3)

%timeit comprehension()
%timeit map_lambda()
%timeit generator()
%timeit for_loop()

250 µs ± 7.59 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
275 µs ± 4.33 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
261 µs ± 2.36 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
289 µs ± 5.33 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


The generator was quicker than the list/map/lambda approach, but still not quicker than the list comprehension. But let's pull the `list()` call out again and see where we are.

In [119]:
thousand = list(range(0, 1000))

def comprehension():
    [x**3 for x in thousand]

def map_lambda():
    map(lambda x: x**3, thousand)

def generator():
    (x**3 for x in thousand)

def for_loop():
    cubes = []
    for x in thousand:
        cubes.append(x**3)

%timeit comprehension()
%timeit map_lambda()
%timeit generator()
%timeit for_loop()

241 µs ± 4.26 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
225 ns ± 0.67 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
310 ns ± 7.36 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
283 µs ± 3.75 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Surprisingly, using `map` and `lambda` is quicker here. This is very close though, in this example it is a different of nanoseconds. This is a very important point to keep in mind.

## Which should I be using?

It matters! You first need to figure out what you are actually trying to solve and if performance is really a concern of yours. These differences here are incredibly subtle micro optimizations, and are most likely going to be unnoticeable. If you really believe that the difference here is going to be impactful, maybe because you run very heavy calculations on very large data sets, very often, you may want to consider moving some work to C. Tiny python micro optimizations may not give you the benefit you are looking for.

The real benefit from this should be that you start considering if you actually need a list or if an iterator will fit your needs. We did see that creating an iterator is significantly faster. Using a generator expression is plenty fast, and reads very nicely in python.